# INSTALL & IMPORTS

In [12]:
#pip install selenium -q
#pip install requests -q
# pip install bs4 -q 

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# TESTING

In [42]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [81]:
page_source = driver.page_source
profile = BeautifulSoup(page_source, 'html.parser')
name = profile.find("span", class_='encounters-story-profile__name').text
age = profile.find("span", class_= "encounters-story-profile__age").text
about = profile.find("ul", class_= "encounters-story-about__badges").text if profile.find("ul", class_= "encounters-story-about__badges") else ""
if about:
    about = re.split('(?=[A-Z])', about)

location = profile.find("span", class_="header-2 text-color-black").text
more_info = profile.find("p", class_="encounters-story-about__text").text
job = profile.find("p", class_="encounters-story-profile__occupation").text
education = profile.find("p", class_="encounters-story-profile__education").text


print(name, age, about, location, more_info, job, education)

Joan , 30 ['174 cm', 'Active', 'Aries', 'Graduate degree', 'Socially', 'Never', 'Man', 'Something casual', 'Open to kids', 'Agnostic'] Barcelona Buscant còmplice per tast de cerveses

That's what she said Advocat Universitat Pompeu Fabra 2015


In [83]:
education

'Universitat Pompeu Fabra 2015'

In [54]:
# Esperar a que el botón esté presente y sea clickeable
try:
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
    )
    button.click()
    print("Botón clickeado con éxito")
except Exception as e:
    print("No se pudo clickear el botón:", e)


Botón clickeado con éxito


# FUNCTION TO TAKE ALL INFO

In [111]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [142]:
def extract_profile_info():

    profiles = []
    prevJob = ""
    prevStories = ""
    prevEducation = ""
    
    for i in range(100):
        
        # Obtener el contenido de la página actual
        page_source = driver.page_source
    
        # Analizar el contenido de la página con BeautifulSoup
        profile = BeautifulSoup(page_source, 'html.parser')
    
    
        # Realizar un diccionario para que la información se vaya guardando.
        profiles_dict = {}
    
        # Esperar 15 segundos antes de extraer la información
        time.sleep(5)
    
        # Extraer información del perfil
        name = profile.find("span", class_='encounters-story-profile__name')
        profiles_dict["name"] = name.text if name else None
        
        age = profile.find("span", class_= "encounters-story-profile__age")
        profiles_dict["age"] = age.text if age else None
        
        about_text = profile.find("ul", class_= "encounters-story-about__badges")
        
        about_text = about_text.text if about_text else None
        about_categories = re.findall('[A-Z][^A-Z]*', about_text) # Se para las palabras cuando detecta una mayúscula. 
        profiles_dict["about"] = ','.join(about_categories)
    
        location = profile.find("span", class_="header-2 text-color-black")
        profiles_dict["location"] = location.text if location else ""
    

        story_about = profile.find("p", class_="encounters-story-about__text")
        profiles_dict["story_about"] = story_about.text if story_about and story_about.text != prevStories else None
        if story_about:
            prevStories = story_about.text

        
        job = profile.find("p", class_="encounters-story-profile__occupation")
        profiles_dict["job"] = job.text if job and job.text != prevJob else None
        if job:
            prevJob = job.text
    
        education = profile.find("p", class_="encounters-story-profile__education")
        profiles_dict["education"] = education.text if education and education.text != prevEducation else None
        if education:
            prevEducation = education.text
    
        # Imprime el diccionario para depuración
        print(profiles_dict)

        profiles.append(profiles_dict)
    
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
            )
            time.sleep(5)  # Añadir un temporizador adicional de 5 segundos
            button.click()
            print("Botón clickeado con éxito")
        except Exception as e:
            print("No se pudo clickear el botón:", e)
            

    return profiles

In [157]:
profile_info = extract_profile_info() 

# Make a DataFrame from a dict. 
df = pd.DataFrame(profile_info)

# Print DataFrame
df

{'name': 'Jelmer', 'age': ', 24', 'about': 'Active,Aries,In grad school,Socially,Never,Relationship,Want someday,Left', 'location': 'Barcelona', 'story_about': 'Guía de kayak, snorkel y cliff diving en la Costa brava, viviendo en barcelona \n🏃\u200d♂️Psicología del deporte\nMe encanta hacer bouldering, yoga, tocar la guitarra, leer y el resto me preguntas si te interesa 🌞\nAvailable in 🇳🇱🇬🇧🇪🇸', 'job': 'Guía De Aventuras', 'education': 'Universitat Autònoma de Barcelona 2024'}
Botón clickeado con éxito
{'name': 'Sebastian', 'age': ', 25', 'about': 'Man,Something casual', 'location': 'Barcelona', 'story_about': None, 'job': None, 'education': None}
Botón clickeado con éxito
{'name': 'Harry', 'age': ', 39', 'about': 'Active,Libra,Graduate degree,Socially,Never,Man,Don’t know yet', 'location': 'Salou, Tarragona', 'story_about': 'Based in sunny Barcelona, meanwhile enjoying occasional cycling in rainy Amsterdam and traveling this beautiful world for a living.', 'job': 'Product, Tech, Entrep

,name,age,about,location,story_about,job,education
0,Jelmer,", 24","Active,Aries,In grad school,Socially,Never,Rel...",Barcelona,"Guía de kayak, snorkel y cliff diving en la Co...",Guía De Aventuras,Universitat Autònoma de Barcelona 2024
1,Sebastian,", 25","Man,Something casual",Barcelona,None,None,None
2,Harry,", 39","Active,Libra,Graduate degree,Socially,Never,Ma...","Salou, Tarragona","Based in sunny Barcelona, meanwhile enjoying o...","Product, Tech, Entrepreneurship etc",University of Groningen 2008
3,Conan,", 29","Active,Gemini,Graduate degree,Socially,Never,M...",Barcelona,Sick of seeing everyone you know taken?\nDo yo...,None,None
4,Carlos,", 52","Socially,Never,Man,Not sure yet,Spiritual,Left","L'Hospitalet de Llobregat, Barcelona","Interesado en la naturaleza, los viajes y las ...",Consultor IT,Universidad de Salamanca
...,...,...,...,...,...,...,...
95,Victor,", 25","Active,Man,Don’t know yet",Barcelona,None,None,None
96,Lucas,", 44","Active,In grad school,Socially,Don’t know yet","Sant Adrià de Besòs, Barcelona","De 🇦🇷 viviendo en 🇪🇸 (Badalona) Soltero, sin h...",None,None
97,Cristobal,", 32","Active,Gemini,Graduate degree,Socially,Man",Barcelona,None,None,None
98,Yail,", 30","Active,Pisces,Graduate degree,Socially,Relatio...","L'Hospitalet de Llobregat, Barcelona",Otro campeón del mundo más en Barcelona 🇦🇷🧉,None,None


In [159]:
df.to_csv('df_final6.csv', index=False)

In [158]:
df.tail(30)

,name,age,about,location,story_about,job,education
70,Francesc,", 31","Active,Aries,Graduate degree,Socially,Man,Some...",Barcelona,🎾🐶🍕☀️🌍🌲🧘📚👨‍💻\n@franquiiiii,IT,None
71,Guillem,", 26","Active,Graduate degree,Socially,Never,Man,Some...",Barcelona,None,None,None
72,Agustin,", 36","Active,Leo,Trade/tech school,Socially,Never,So...","Barcelona El Prat Airport, Barcelona","Godin de día, maestro de baile y DJ por las no...",None,None
73,Roger,", 27","Active,Cancer,Undergraduate degree,Socially,So...",Barcelona,None,None,None
74,Andrea,", 28","Active,Socially,Man,Don’t know yet",Barcelona,Delu.andre\n🇪🇸🇮🇹🇬🇧\nFrom Florence,Account Manager at Google,None
75,Andrés,", 32","Active,Libra,Graduate degree,Socially,Never,Man",Barcelona,Viajar!,None,None
76,Ole,", 24","Active,Undergraduate degree,Socially,Man,Want ...",Barcelona,"Aqui por 3 meses, \nMe muestras tu lugar favor...",Barkeeper,None
77,Yassir,", 36","Active,Leo,Graduate degree,Socially,Socially,M...",Barcelona,Conocer y fluir con alguien guay y divertida 🥳🥳,Senior Manager at Vendor de humo en Consultorí...,Economia
78,Arthur,", 31","Sometimes,Gemini,Graduate degree,Socially,Soci...",Barcelona,Frenchy working in Tech industry. 👨‍💻\nA bit m...,Building EMEA Teams at Global Stealth Startup,Skema
79,Orfeo,", 41","Active,Graduate degree,Socially,Man,Don’t know...",Barcelona,Enamorado de la vida y de la gente con buen co...,None,EU BBA y UB Filosofia
